<div >
<img src = "figs/banner.png" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/UsoIA/blob/main/01_Intro_Notebook_LLM.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Chat GPT: Entendiendo los Modelos de Lenguaje y su Funcionamiento Interno

Chat GPT es lo que llamamos un **modelo de lenguaje**, ya que se encarga de modelar secuencias de palabras, caracteres o, de forma más general, **tokens**. Su principal habilidad radica en comprender cómo las palabras se relacionan y siguen unas a otras en un idioma, en este caso, el inglés.

Desde su perspectiva, lo que realmente hace es **completar secuencias**: tú le proporcionas el inicio de una secuencia y el modelo genera el resto basándose en patrones que ha aprendido durante su entrenamiento. Por esta razón, se clasifica como un modelo de lenguaje.

Sin embargo, más allá de esta funcionalidad básica, es importante explorar cómo funciona "bajo el capó" Chat GPT, es decir, entender los componentes internos que hacen posible su desempeño. La base de este modelo es una red neuronal que modela la secuencia de palabras, y su funcionamiento se deriva de un artículo científico clave titulado **"Attention is All You Need"** publicado en 2017. Este trabajo marcó un antes y un después en la inteligencia artificial al introducir la **arquitectura Transformer**, la piedra angular de sistemas como Chat GPT.

<div >
<img src = "figs/transformer.png" />
</div>

El término GPT significa **"Generative Pre-trained Transformer"** (Transformer Generativo Preentrenado), destacando los tres elementos clave que lo definen:

1. **Generative (Generativo):** Es capaz de generar texto de forma autónoma y coherente.
2. **Pre-trained (Preentrenado):** Ha sido entrenado previamente con grandes cantidades de datos para entender patrones lingüísticos.
3. **Transformer:** Utiliza la arquitectura Transformer, que basa su eficacia en un mecanismo conocido como **atención** para procesar y modelar secuencias de texto con gran precisión.

Este tutorial se centrará en desglosar cómo la arquitectura Transformer hace posible que Chat GPT modele las secuencias de lenguaje y produzca respuestas.

In [3]:
import urllib.request


# read it in to inspect it
url = 'https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt'
with urllib.request.urlopen(url) as f:
    text = f.read().decode('utf-8')  # Decode the bytes to string


In [4]:
# print el numero de caracters unicos en text
print(len(text))

1038397


In [5]:
print(text[:1000])

DON QUIJOTE DE LA MANCHA
Miguel de Cervantes Saavedra

PRIMERA PARTE
CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha
En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lentejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas con sus pantuflos de lo mismo, los días de entre semana se honraba con su vellori de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años, era de complexión recia, sec

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))



 !"'(),-.0123456789:;<?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijlmnopqrstuvxyz¡«»¿̀́̃̈–‘’“”


In [7]:
print(vocab_size)

89


In [8]:
stoi = { ch:i for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]

In [10]:
print(encode("Hola Don"))

[31, 65, 62, 52, 1, 27, 65, 64]


In [11]:
itos = { i:ch for i,ch in enumerate(chars) }
decode = lambda l: ''.join([itos[i] for i in l])

In [12]:
print(decode(encode("Hola Don")))

Hola Don


In [13]:
import torch

In [14]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1038397]) torch.int64
tensor([27, 38, 37,  1, 40, 44, 32, 33, 38, 43, 28,  1, 27, 28,  1, 35, 24,  1,
        36, 24, 37, 26, 31, 24,  0, 36, 60, 58, 71, 56, 62,  1, 55, 56,  1, 26,
        56, 68, 72, 52, 64, 70, 56, 69,  1, 42, 52, 52, 72, 56, 55, 68, 52,  0,
         0, 39, 41, 32, 36, 28, 41, 24,  1, 39, 24, 41, 43, 28,  0, 26, 24, 39,
        32, 81, 43, 44, 35, 38,  1, 11, 20,  1, 40, 71, 56,  1, 70, 68, 52, 70,
        52,  1, 55, 56,  1, 62, 52,  1, 54, 65, 64, 55, 60, 54, 60, 65, 81, 64,
         1, 74,  1, 56, 61, 56, 68, 54, 60, 54, 60, 65,  1, 55, 56, 62,  1, 57,
        52, 63, 65, 69, 65,  1, 59, 60, 55, 52, 62, 58, 65,  1, 27,  9,  1, 40,
        71, 60, 61, 65, 70, 56,  1, 55, 56,  1, 62, 52,  1, 36, 52, 64, 54, 59,
        52,  0, 28, 64,  1, 71, 64,  1, 62, 71, 58, 52, 68,  1, 55, 56,  1, 62,
        52,  1, 36, 52, 64, 54, 59, 52,  7,  1, 55, 56,  1, 54, 71, 74, 65,  1,
        64, 65, 63, 53, 68, 56,  1, 64, 65,  1, 67, 71, 60, 56, 68, 65,  1, 52,
      

In [15]:
# Dividir en entrenamiento y validation set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
val_data

tensor([62, 60, 53,  ..., 32, 42,  0])

In [16]:
block_size = 8
train_data[:block_size+1]

tensor([27, 38, 37,  1, 40, 44, 32, 33, 38])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"cuando el insumo es (x) {context} el objetivo (y): {target}")

cuando el insumo es (x) tensor([27]) el objetivo (y): 38
cuando el insumo es (x) tensor([27, 38]) el objetivo (y): 37
cuando el insumo es (x) tensor([27, 38, 37]) el objetivo (y): 1
cuando el insumo es (x) tensor([27, 38, 37,  1]) el objetivo (y): 40
cuando el insumo es (x) tensor([27, 38, 37,  1, 40]) el objetivo (y): 44
cuando el insumo es (x) tensor([27, 38, 37,  1, 40, 44]) el objetivo (y): 32
cuando el insumo es (x) tensor([27, 38, 37,  1, 40, 44, 32]) el objetivo (y): 33
cuando el insumo es (x) tensor([27, 38, 37,  1, 40, 44, 32, 33]) el objetivo (y): 38


In [18]:
torch.manual_seed(1337)
block_size = 8 # tamaño de contexto
batch_size = 4 # numero de secuencias en paralelo

In [19]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [20]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[70, 52, 55,  1, 55, 56,  1, 66],
        [ 7,  1, 54, 65, 64,  1, 56, 62],
        [60,  1, 52,  1, 63, 60, 81,  1],
        [54, 52, 68, 52,  7,  1, 64, 60]])
targets:
torch.Size([4, 8])
tensor([[52, 55,  1, 55, 56,  1, 66, 52],
        [ 1, 54, 65, 64,  1, 56, 62,  1],
        [ 1, 52,  1, 63, 60, 81,  1, 63],
        [52, 68, 52,  7,  1, 64, 60,  1]])


In [21]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [22]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [23]:
m = BigramLanguageModel(vocab_size)
logits = m(xb, yb)

In [24]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


8sLv<.OTM»̃"DK;C??c"TŃ6)-HpLL);GhLRN!̀’IJ1PS«L¡X9”:l";;]:5Vnd»PjYV“Y«"ITaQ2’̀rE?x”b‘5”c¿o'H7L[cB3;Q


In [25]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [26]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.233574628829956


In [27]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


N: caro, SAle dono Sa el ]Qunusa es sanzcompomi nte dida la yasueriela W¡cha tre eIAmo vondiece os do cos as horos, da hacise Q ja yen pe. anteta ta, s, pas lomes cárá micora; yobiesañodabrel, pé, ve sís al dí tenójo acovidrlllo e leno vira se sosan queguerigo, de ce –X dejon cor sna, ba durídenta l o; nsOre s scicotea labla dílo y EVX(Anazoronte pesalo, Dosabra desin te bíncú tadabustométadera cegurtená la po a biase deblane y, cahe, litapamprcimide brama e canena ale s si¿mon lo, 
